### `tes-thermo`
---

In this notebook, we will use `tes-thermo` to calculate the equilibrium compositions for the steam methane reforming (SMR) process.

Version 0.1.1 of `tes-thermo` allows the user to leverage the `thermo` library to retrieve information about the components and also enables the addition of new components.

First, import the Component and Gibbs classes. Then, specify the components to be considered.

In [ ]:
%pip install -qU tes-thermo

In [6]:
from tes_thermo.utils import Component
from tes_thermo.gibbs import Gibbs
import numpy as np

new_components = {
        "methane": {
            "name": "methane",
            "Tc": 190.6, "Tc_unit": "K",
            "Pc": 45.99, "Pc_unit": "bar",
            "omega": 0.012,
            "Vc": 98.6, "Vc_unit": "cm³/mol",
            "Zc": 0.286,
            "Hfgm": -74520, "Hfgm_unit": "J/mol",
            "Gfgm": -50460, "Gfgm_unit": "J/mol",
            "structure": {"C": 1, "H": 4},
            "phase": "g",
#            "kijs": [0, 0, 0, 0, 0, 0], # Interaction parameters with other components
            "cp_polynomial": lambda T: 8.314 * (1.702 + 0.009081* T -0.000002164*T**2),
        }
    }

components = ['water','carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol']

In `new_components`, the components to be added are defined. In this case, the user must specify all the thermodynamic properties of the component as well as the polynomial used to calculate $C_p$.

For this example, the following polynomial was used:

$$C_p(T) = R \times \left( 1.702 + 0.009081T - 0.000002164T^2 \right)$$

where $T$ is the temperature in Kelvin and $ C_p $ is the heat capacity in J/(mol·K).

The components listed in `components` will be instantiated using the `Chemical` class from the `thermo` library.

In [7]:
comps = Component(components, new_components)
comps = comps.get_components()
gibbs = Gibbs(components=comps,equation='Peng-Robinson')
res = gibbs.solve_gibbs(T=800, T_unit= 'K',
                         P=60, P_unit='bar',
                         initial=np.array([0, 1, 0, 0, 1, 0]))

2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Initializing Gibbs class...
2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Components created as Chemical from the thermo library: ['water', 'carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol']
2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Manually defined components: [{'name': 'methane', 'Tc': 190.6, 'Pc': 4599000.0, 'omega': 0.012, 'Vc': 9.859999999999998e-05, 'Zc': 0.286, 'Hfgm': -74520.0, 'Gfgm': -50460.0, 'phase': 'g', 'structure': {'C': 1, 'H': 4}, 'kijs': None, 'cp_coeffs': None, 'cp_polynomial': <function <lambda> at 0x326631760>}]
2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Equation of state: Peng-Robinson
2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Calculated kijs for methane: [0.013270068977719762, 0.00018971524403865736, 9.015419903968258e-05, 0.007480048013087104, 0.0008589821936367814, 0.0]
2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Starting Gibbs minimization probl

[-216510.76161070535, -190698.06341194085, -409037.01549091557, -8427.091101173908, -115109.93596023758, -22486.752311312306]


2025-08-15 00:36:31 - tes_thermo.gibbs.gibbs - INFO - Optimal solution found successfully


Note that the `Component` class takes both `components` and `new_components` as arguments. The `get_components` method already returns the required structure to instantiate the `Gibbs` class.

With this information, the `solve_gibbs` method returns the equilibrium compositions at temperature $T$ and pressure $P$, given the initial compositions specified by `initial`.

In [8]:
res

{'Temperature (K)': 800.0,
 'Pressure (bar)': 60.00000000000001,
 'Water': 0.04337941510960611,
 'Carbon monoxide': 0.11003626275025695,
 'Carbon dioxide': 0.923292149916935,
 'Hydrogen': 0.023277493427407873,
 'Methanol': 5.225802852767064e-08,
 'Methane': 0.9666715055286526}